## Import Libraries

In [13]:

import ee
import geemap as gee
import geemap.chart as chart
import pandas as pd
import os

## Create an Interactive MAp

In [4]:
ke = gee.Map(center=(0.0236, 37.9062),zoom=7)
ke

Map(center=[0.0236, 37.9062], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDat…

## Import Feature Collection 
   - Farm centroid Locations
   - Add them to the Map

In [5]:
# Define the AOS_2022_all feature collection
AOS_2022_all = ee.FeatureCollection("projects/ee-koroshillary1212/assets/AOS_2022_all")
# Check the size of the FeatureCollection
print('Size of AOS_2022_all:', AOS_2022_all.size().getInfo())
# Get a list of all values in the County property
county_list = AOS_2022_all.aggregate_array('County').distinct().getInfo()
# Print the unique counties
print('Unique counties:', county_list)
# Add fatures to the Map
ke.addLayer(AOS_2022_all, {}, "AOS_2022_all")
ke

Size of AOS_2022_all: 7597
Unique counties: ['Nandi', 'Nakuru', 'Trans Nzoia', 'Tharaka Nithi', 'Kilifi', 'Makueni', 'Bungoma', 'Kwale', 'Kitui', 'Kakamega', 'Machakos', 'Embu', 'Taita Taveta']


Map(bottom=16682.0, center=[0.0236, 37.9062], controls=(WidgetControl(options=['position', 'transparent_bg'], …

In [6]:
df = gee.ee_to_df(AOS_2022_all)
df

,County,Farmer’s,HHCoordina,IDNumber,MobileNumb,Respondent,SubCounty,Village,Ward,altitude,latitude,longitude,precision
0,Nandi,David Kiprop,0.0152224 34.9664602 1830.6999999999998 3.0,34349914,700238378,Control,Aldai,Kipsis Village,Kobujoi,1830.7,0.015222,34.966460,3.000
1,Nandi,Peter Melly,0.0425617 34.9762867 1898.3999999999999 2.1,5589785,113373869,Control,Aldai,Kipsis Village,Kobujoi,1898.4,0.042562,34.976287,2.100
2,Nandi,Wilson Rop,0.0166837 34.9633845 1842.8 2.733,16093779,710294849,Control,Aldai,Kipsis,Kobujoi,1842.8,0.016684,34.963384,2.733
3,Nandi,Margaret Lagat,0.013196 34.9675127 1815.8 2.8,3287825,704353656,Control,Aldai,Kipsis Village,Kobujoi,1815.8,0.013196,34.967513,2.800
4,Nandi,Peres Jepkemboi,0.0060558 34.9751345 1802.6999999999998 2.3,24982843,707297721,Control,Aldai,Kipsis Village,Kobujoi,1802.7,0.006056,34.975135,2.300
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7592,Machakos,Luisa mumbua kilonzo,-1.0845443 37.4450167 1158.6 3.62,27654617,710954192,Treatment,Yatta,Vondeni,Ndalani,1158.6,-1.084544,37.445017,3.620
7593,Machakos,Sammy Ngumbau,-1.0845433 37.4450183 1158.6 3.9,1900941,726617406,Treatment,Yatta,Kwakulu,Ndalani,1158.6,-1.084543,37.445018,3.900
7594,Machakos,THERESIA Ndunge,-1.0848133 37.4448847 1218.9 10.125,3420218,707391392,Treatment,Yatta,Matithi,Ndalani,1218.9,-1.084813,37.444885,10.125
7595,Machakos,Samuel mwaura mwololo,-1.0841343 37.4434327 1385.2 27.645,25822555,704488540,Treatment,Yatta,Matithi,Ndalani,1385.2,-1.084134,37.443433,27.645


## Load Satellite ImageCollection

In [7]:
# Specify the county you want to select
county_name = 'Tharaka Nithi'  # Replace with your specific county

# Filter features by county
county_features = AOS_2022_all.filter(ee.Filter.eq('County', county_name))

# Print the selected features
print('Features in {}:'.format(county_name), county_features.size().getInfo())

# Display filtered collection as default and with a custom color
ke.centerObject(county_features, 9)

# Import the Landsat 9 Level 2, Collection 2, Tier 1 image collection
l9 = (ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')
      .filterBounds(county_features.geometry())
      .filterDate('2022-01-01', '2022-12-31')
      .sort('CLOUD_COVER')
      .select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'ST_B10']))

# Apply scaling factors
def apply_scale_factors(image):
    optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0).subtract(273.15)
    return image.addBands(optical_bands, None, True).addBands(thermal_bands, None, True)

l9 = l9.map(apply_scale_factors)

# Functions to add indices
def add_ndvi(image):
    ndvi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')
    return image.addBands(ndvi)

def add_evi(image):
    evi = image.expression(
        '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))',
        {
            'NIR': image.select('SR_B5'),
            'RED': image.select('SR_B4'),
            'BLUE': image.select('SR_B2')
        }).rename('EVI')
    return image.addBands(evi)

def add_ndwi(image):
    ndwi = image.expression(
        '(GREEN - NIR) / (GREEN + NIR)',
        {
            'GREEN': image.select('SR_B3'),
            'NIR': image.select('SR_B5')
        }).rename('NDWI')
    return image.addBands(ndwi)

def add_ipvi(image):
    ipvi = image.expression(
        'NIR / (NIR + RED)',
        {
            'NIR': image.select('SR_B5'),
            'RED': image.select('SR_B4')
        }).rename('IPVI')
    return image.addBands(ipvi)

def add_osavi(image):
    osavi = image.expression(
        '(NIR - RED) / (NIR + RED + 0.16)',
        {
            'NIR': image.select('SR_B5'),
            'RED': image.select('SR_B4')
        }).rename('OSAVI')
    return image.addBands(osavi)

def add_gndvi(image):
    gndvi = image.expression(
        '(NIR - GREEN) / (NIR + GREEN)',
        {
            'NIR': image.select('SR_B5'),
            'GREEN': image.select('SR_B3')
        }).rename('GNDVI')
    return image.addBands(gndvi)

# Map over the image collection and add all indices to each image
l9 = l9.map(add_ndvi).map(add_evi).map(add_ndwi).map(add_ipvi).map(add_osavi).map(add_gndvi)

# Get the least cloudy image for display
image = l9.sort('CLOUD_COVER').first()

# Display the result
ke.centerObject(image, 9)

# Visualization Parameters
styled_bgfm = AOS_2022_all.style(color='ff5d00', pointSize=7.0)

ndvi_params = {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']}
temp_params = {'min': -11, 'max': 43, 'palette': ['blue', 'yellow', 'red']}
osavi_params = {'min': -1, 'max': 1, 'palette': ['grey', 'yellow', 'blue']}
ipvi_params = {'min': -1, 'max': 1, 'palette': ['white', 'yellow', 'blue']}

ke.addLayer(image.select('NDVI'), ndvi_params, 'NDVI image', False)
ke.addLayer(image.select('EVI'), ndvi_params, 'EVI image', False)
ke.addLayer(image.select('NDWI'), ndvi_params, 'NDWI image', False)
ke.addLayer(image.select('OSAVI'), osavi_params, 'OSAVI image', True)
ke.addLayer(image.select('GNDVI'), ndvi_params, 'GNDVI image', False)
ke.addLayer(image.select('IPVI'), ipvi_params, 'IPVI image', False)
ke.addLayer(image.select('ST_B10'), temp_params, 'ST_B10 image', False)
ke.addLayer(styled_bgfm, {}, 'Colored - Filtered')
ke

Features in Tharaka Nithi: 526


Map(bottom=16682.0, center=[-0.003352174682753846, 37.41178846846008], controls=(WidgetControl(options=['posit…

In [8]:
county_features

In [9]:
# Randomly select a single feature from the BGFM feature collection
test_point = ee.Feature(county_features.first())
test_point_name = test_point.getString("Farmer’s").getInfo()

print('Test Point Name:', test_point_name)


Test Point Name: Geoffrey murithi


In [11]:
# Map over the image collection; for each image find the intersecting points and do a region reduction at each point
def compute_stats(img):
    img_date = img.date().format('YYYY-MM-dd')
    img_id = img.get('LANDSAT_ID')
    img_sat = img.get('SATELLITE')
    pts = county_features.filterBounds(img.geometry())
    band_stats = img.reduceRegions(
        collection=pts,
        reducer=ee.Reducer.mean(),
        scale=30,
        crs='EPSG:4326'
    ).map(lambda pt: pt.set({
        'imgDate': img_date,
        'imgId': img_id,
        'imgSat': img_sat,
        'plotId': pt.get('PlotID')
    }))
    return band_stats

stat_col = l9.map(compute_stats).flatten()
stat_col = stat_col.filter(ee.Filter.notNull(['NDVI', 'EVI', 'NDWI', 'IPVI', 'OSAVI', 'GNDVI', 'ST_B10']))




df_stat_col=gee.ee_to_df(stat_col)
# Print a sample of the points
df_stat_col.size

343224

In [14]:
 #Export to local directory
output_dir = 'data'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

output_file = os.path.join(output_dir, 'AOS2022_Landsat_{}.csv'.format(county_name))
df_stat_col.to_csv(output_file, index=False)

print(f"Data exported to {output_file}")


Data exported to data\AOS2022_Landsat_Tharaka Nithi.csv


In [18]:
# Read the exported CSV file using pandas
df = pd.read_csv(output_file)
df.head()

,County,EVI,Farmer’s,GNDVI,HHCoordina,IDNumber,IPVI,MobileNumb,NDVI,NDWI,...,SR_B7,ST_B10,SubCounty,Village,Ward,altitude,imgDate,latitude,longitude,precision
0,Tharaka Nithi,0.183500,Geoffrey murithi,0.454344,-0.2667674 37.8384805 708.702 3.9,2415380,0.655537,707509554,0.311075,-0.454344,...,0.221658,47.403842,Igambangombe,Kanthaje,Igambangombe,708.702,2022-03-05,-0.266767,37.838481,3.9
1,Tharaka Nithi,0.183500,Beatrice muthoni,0.454344,-0.2667981 37.8385407 735.796 3.9,28193438,0.655537,701603931,0.311075,-0.454344,...,0.221658,47.403842,Igambangombe,Kanthaje,Igambangombe,735.796,2022-03-05,-0.266798,37.838541,3.9
2,Tharaka Nithi,0.301561,Simon kanampiu kithinji,0.583619,-0.2819921 37.841794 850.342 3.9,2469234,0.742496,712572895,0.484992,-0.583619,...,0.163935,43.811503,Igambangombe,Kanthaje,Igambangombe,850.342,2022-03-05,-0.281992,37.841794,3.9
3,Tharaka Nithi,0.287107,Japhet njeru riungu,0.596586,-0.269033 37.8368785 750.121 3.9,22005107,0.750639,702109594,0.501277,-0.596586,...,0.144300,46.337420,Igambangombe,Kanthaje,Igambangombe,750.121,2022-03-05,-0.269033,37.836878,3.9
4,Tharaka Nithi,0.309406,George karigu mugwika,0.565327,-0.2695918 37.8339065 758.635 3.9,11056910,0.740941,703755216,0.481883,-0.565327,...,0.183982,45.674324,Igambangombe,Kanthaje,Igambangombe,758.635,2022-03-05,-0.269592,37.833906,3.9
